In [39]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from scipy.stats import invgamma
import math

In [40]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [41]:
class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedforwardNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size) #bias = False?
        self.relu1 = nn.ReLU()
        self.output_layer = nn.Linear(hidden_size, output_size)
        self.lastlayeractivation = None

    def forward(self, x):
        x = self.relu1(self.layer1(x))
        last_hidden_activation = x
        x = self.output_layer(x)
        return last_hidden_activation, x

# Set random seed for reproducibility
torch.manual_seed(42)

# Instantiate the neural network
input_size = 20
hidden_size = 20
output_size = 1
model = FeedforwardNN(input_size, hidden_size, output_size)

# # Forward pass
# output = model_1(synthetic_input)

# To GPU
# model_1 = model_1.to(device)

# Print the architecture and output shape
print(model_1)
# print("Output shape:", output.shape)

FeedforwardNN(
  (layer1): Linear(in_features=20, out_features=20, bias=True)
  (relu1): ReLU()
  (output_layer): Linear(in_features=20, out_features=1, bias=True)
)


In [143]:
T = 100
d = 20
N = 2
capacity = 50
L = 10

In [119]:
synthetic_input = torch.randn(T,d)
synthetic_input[0]

tensor([ 0.5340, -1.4064,  0.1673, -2.1951, -0.3262, -0.0105, -0.0137,  0.3387,
        -0.7112,  1.3265, -1.7022,  1.9739, -2.0169,  0.5995, -1.7559, -1.2022,
         1.2553, -0.6220,  0.4855, -1.4063])

In [120]:
def random_ball(num_points,dimension = 20,radius = 1):
    output = []
    for n in range(num_points):
        x = np.random.randn(dimension)
        x /= np.linalg.norm(x, ord=2)
        x = torch.tensor(x,dtype = torch.float32)
        if num_points == 1:
            return x
        output.append(x)
    return torch.stack(output)

In [123]:
# Careful: Adding too much noise breaks it as the noise takes over. But noise doesn't break if the reward is high

a = random_ball(1,20,1).reshape(-1)
def reward_function(context):
    return 10*(torch.dot(context,a)**2).item() + (0.1*torch.randn(1)).item()
reward_function_quadratic(random_ball(1,20))

1.025502499192953

In [145]:
replay_buffer = [] #E in algorithm

mu_hat = np.zeros((N,d))
mu_initial = np.zeros((N,d))

phi_initial = []
phi = []
for i in range(N):
    phi_initial.append(np.eye(d))
    phi.append(np.zeros((d,d)))
phi = np.array(phi)
phi_initial = np.array(phi_initial)
f = np.zeros((N,d))
A = np.ones(N)
B = np.ones(N)
B_initial = np.ones(N)
R_2 = np.zeros(N)
b = np.zeros(N)

for t in range(T):
    context = synthetic_input[t]
    last_layer, estimated_reward = model(context)
    v = np.zeros(N)
    mu = np.zeros((N,d))
    for i in range(N):
        v[i] = invgamma.rvs(A[i],scale=B[i]).item()
        v_i_squared = v[i].item()**2
        cov_matrix = v_i_squared*(phi_initial[i] + phi[i])
        mu[i] = np.random.multivariate_normal(mean=mu_hat[i], cov=cov_matrix)  
    last_layer = last_layer.detach().numpy()
    arm = np.argmax(last_layer@mu.T)
    reward = reward_function(context)
    if len(replay_buffer) == capacity:
        for i in range(capacity):
            if replay_buffer[i][1] == arm:
                replay_buffer.pop(i)
                break
    replay_buffer.append((context,arm,reward))
    phi[arm] = phi[arm] + last_layer@last_layer.T
    f[arm] = f[arm] + last_layer*reward
    mu_hat[arm] = np.linalg.inv(phi_initial[arm]+phi[arm]) @ (phi_initial[arm]@mu_initial[arm]+f[arm])
    A[arm] = A[arm] + 0.5
    R_2[arm] = R_2[arm] + reward**2
    x = mu_initial[arm].T@phi_initial[arm]@mu_initial[arm]
    y = mu_hat[arm].T@phi[arm]@mu_hat[arm]
    B[arm] = B_initial[arm] + 0.5*(R_2[arm]+x-y)
    print(B[arm])
    
    # To be updated:
    if (t+1)%L == 0:
        for i in range(N):
            pass
        pass
    break




397.95811735192996
